In [1]:
import os
import pickle
import re
import math
import csv 
import string
import unidecode
import numpy as np
import pandas as pd
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import PlaintextCorpusReader

[nltk_data] Downloading package stopwords to /home/nimit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Root folder where the cleaned text files to be located
corpus_root = '/home/nimit/IITK/IR_CS657A/Assign1/Solution/english-corpora-processed'


In [ ]:
# Read the list of files
filelists = PlaintextCorpusReader(corpus_root, '.*')

In [ ]:
# List down the IDs of the files read from the local storage
doc_list=filelists.fileids()

In [ ]:
corpus_size=len(filelists.fileids())

In [ ]:
unique_words_dict = {}

In [ ]:
doc_normalised_length=np.zeros(corpus_size)

In [ ]:
doc_len_list=[]
global_matrix=[]

In [ ]:
i=0

for filename in filelists.fileids():
    index = defaultdict(int)
    with open(corpus_root+'/'+str(filename),'r+')as f:
        text=f.read()
        tokens_words=nltk.word_tokenize(text)
        for word in tokens_words[1:-1]: 
            index[word]+=1
        
    doc_len_list.append(len(index))
    global_matrix.append(index)
    for item in index:
        doc_normalised_length[i]+=((1+np.log(index[item]))**2)
        try:
            unique_words_dict[item][(str(filename).replace('.txt',''))]=index[item]
        except:
            unique_words_dict[item]={}
            unique_words_dict[item][(str(filename).replace('.txt',''))]=index[item]
    i+=1
    

In [ ]:
with open('/home/nimit/IITK/IR_CS657A/Assign1/Solution/posting_dict.pkl', 'wb') as f:
    pickle.dump(unique_words_dict, f)

In [ ]:
with open('/home/nimit/IITK/IR_CS657A/Assign1/Solution/doc_len_list.pkl', 'wb') as f:
    pickle.dump(doc_len_list, f)

In [ ]:
with open('/home/nimit/IITK/IR_CS657A/Assign1/Solution/global_matrix.pkl', 'wb') as f:
    pickle.dump(global_matrix, f)

In [ ]:
with open('/home/nimit/IITK/IR_CS657A/Assign1/Solution/doc_normalised_length.pkl', 'wb') as f:
    pickle.dump(doc_normalised_length, f)

In [ ]:
with open('/home/nimit/IITK/IR_CS657A/Assign1/Solution/posting_dict.pkl', 'rb') as f:
    loaded_posting_dict = pickle.load(f)

In [ ]:
with open('/home/nimit/IITK/IR_CS657A/Assign1/Solution/doc_len_list.pkl', 'rb') as f:
    loaded_doc_len_list = pickle.load(f)

In [ ]:
with open('/home/nimit/IITK/IR_CS657A/Assign1/Solution/global_matrix.pkl', 'rb') as f:
    loaded_global_matrix = pickle.load(f)

In [ ]:
with open('/home/nimit/IITK/IR_CS657A/Assign1/Solution/doc_normalised_length.pkl', 'rb') as f:
    loaded_doc_normalised_length=pickle.load(f)

In [ ]:
unique_words_all = len(loaded_posting_dict)

In [ ]:
def boolean_query(query):
    
    boolean_result = []
    binary_array_per_word=[]
    binary_array_all_words=[]
    
    for term in query:
        if term in loaded_posting_dict:
            binary_array_per_word=[0]*corpus_size
            temp_dict=loaded_posting_dict[term]
            
            for item in temp_dict:
                temp2=doc_list.index((str(item)+".txt"))
                binary_array_per_word[temp2]=1
            
            binary_array_all_words.append(binary_array_per_word)
        else:
            print(word," not found")
    
    for j in range(len(query)-1):
        word1=binary_array_all_words[0]
        word2=binary_array_all_words[1]

        bitwise_op = [w1 & w2 for (w1,w2) in zip(word1,word2)]
            
        binary_array_all_words.remove(word1)
        binary_array_all_words.remove(word2)
        binary_array_all_words.insert(0,bitwise_op)
  
    res = binary_array_all_words[0]
    cnt = 0
    
    for item in res:
        if item==1:
            boolean_result.append(doc_list[cnt])
        cnt = cnt+1
    
    return boolean_result

In [ ]:
def tf_idf_score(query):
    len_query=len(query)
    tf_query={}
    arr=np.zeros((len_query+1,corpus_size))
    for i in range(len_query):
        try:
            tf_query[query[i]]+=1
        except:
            tf_query[query[i]]=1
            
        temp20 = loaded_posting_dict[query[i]]
        
        for item in temp20:
            temp=item
            temp2=doc_list.index((str(temp)+".txt"))
            #arr[i][temp2]=1+np.log(temp20[item])
            arr[i][temp2]=(temp20[item])
            arr[-1][temp2]+=arr[i][temp2]**2
    
    
    # Processing Query
    query_square=0
    for item in tf_query:
        #tf_query[item]=1+np.log(tf_query[item])
        tf_query[item]=tf_query[item]*(np.log(corpus_size/len(loaded_posting_dict[item]))) #need to handle this case when item is not in dict
        query_square+=tf_query[item]**2
    
    #query_root=math.sqrt(query_square)
    q_arr=np.zeros(len_query)
    
    y=0
    for z in tf_query:
        q_arr[y]=tf_query[z]
        y+=1
        #/query_root
    
    # Final similarity check
    
    doc_score={}
    for i in range(corpus_size):
        s=0
        for j in range(len_query):
            s+=(arr[j][i]*q_arr[j])
        doc_score[doc_list[i]]=s
    doc_score = {k: v for k, v in sorted(doc_score.items(), key=lambda x: x[1],reverse=True)}
    return doc_score


In [ ]:
def BM25(query,k1=1.2,b=0.75):
    
    bm25_scores = {}
    #calculating avg_doc_len value
    avg_doc_len_ = sum(loaded_doc_len_list) / corpus_size
    
    for index in range(corpus_size):
        scr = 0.0
        doc_len = loaded_doc_len_list[index]
        term_frequency = loaded_global_matrix[index]

        for term in query:
            if term not in term_frequency:
                continue
            
            count=len(loaded_posting_dict[term])
            freq = term_frequency[term]
            idf_val=np.log(1 + (corpus_size - count + 0.5) / (count + 0.5))
            num = idf_val * freq * (k1 + 1)
            deno = freq + k1 * (1 - b + b * doc_len / avg_doc_len_)
            scr += (num/deno)
        
        bm25_scores[doc_list[index]]=scr
    
    final_ans=dict(sorted(bm25_scores.items(), key=lambda item: item[1],reverse=True))
    return final_ans